In [1]:
import numpy as np
import pandas as pd
import os
from glob import glob

In [5]:
tcsvs = glob("Temp-csvs/base-data/*.csv")
print(tcsvs)

['Temp-csvs/base-data/ROCKHAMPTON_MaxT_39083.csv', 'Temp-csvs/base-data/ARMIDALE_MinT_56037.csv', 'Temp-csvs/base-data/CAPE-NELSON_MaxT_90184.csv', 'Temp-csvs/base-data/TOWNSVILLE_MinT_32040.csv', 'Temp-csvs/base-data/MORWELL_MaxT_85280.csv', 'Temp-csvs/base-data/BRISBANE_MinT_40913.csv', 'Temp-csvs/base-data/MELBOURNE_MinT_86338.csv', 'Temp-csvs/base-data/SYDNEY_MaxT_66212.csv', 'Temp-csvs/base-data/ADELAIDE_MaxT_23000.csv', 'Temp-csvs/base-data/CANBERRA_MaxT_70351.csv', 'Temp-csvs/base-data/HOBART_MinT_94029.csv', 'Temp-csvs/base-data/LAUNCESTON_MaxT_91237.csv', 'Temp-csvs/base-data/DUBBO_MaxT_65070.csv', 'Temp-csvs/base-data/PORT-AUGUSTA_MaxT_18201.csv', 'Temp-csvs/base-data/CAPE-NELSON_MinT_90184.csv', 'Temp-csvs/base-data/ARMIDALE_MaxT_56037.csv', 'Temp-csvs/base-data/ROCKHAMPTON_MinT_39083.csv', 'Temp-csvs/base-data/TOWNSVILLE_MaxT_32040.csv', 'Temp-csvs/base-data/MELBOURNE_MaxT_86338.csv', 'Temp-csvs/base-data/BRISBANE_MaxT_40913.csv', 'Temp-csvs/base-data/MORWELL_MinT_85280.csv

<font color = 'purple'><font size = 4>
    Am going to merge together all these individual files which have min or max temperature records

In [7]:
file = tcsvs[0]
name = file.split('/')[-1].split(".")[0]
print(name)

ROCKHAMPTON_MaxT_39083


In [9]:
fdf_file = pd.read_csv(file, index_col=0)
fdf_file.head()

,Product code,Bureau of Meteorology station number,Year,Month,Day,Maximum temperature (Degree C),Days of accumulation of maximum temperature,Quality
0,IDCJAC0010,39083,1939,1,1,NaN,NaN,NaN
1,IDCJAC0010,39083,1939,1,2,NaN,NaN,NaN
2,IDCJAC0010,39083,1939,1,3,NaN,NaN,NaN
3,IDCJAC0010,39083,1939,1,4,NaN,NaN,NaN
4,IDCJAC0010,39083,1939,1,5,NaN,NaN,NaN


In [10]:
fdf_file['Date'] = fdf_file.apply(func=(lambda row: pd.to_datetime(f"{row['Year']}-{row['Month']}-{row['Day']}")     ) ,axis=1)

In [11]:
fdf_file.set_index('Date', drop=True, inplace=True)
fdf_file.head()

,Product code,Bureau of Meteorology station number,Year,Month,Day,Maximum temperature (Degree C),Days of accumulation of maximum temperature,Quality
Date,,,,,,,,
1939-01-01,IDCJAC0010,39083,1939,1,1,NaN,NaN,NaN
1939-01-02,IDCJAC0010,39083,1939,1,2,NaN,NaN,NaN
1939-01-03,IDCJAC0010,39083,1939,1,3,NaN,NaN,NaN
1939-01-04,IDCJAC0010,39083,1939,1,4,NaN,NaN,NaN
1939-01-05,IDCJAC0010,39083,1939,1,5,NaN,NaN,NaN


In [13]:
col_name = name #+ "_" + str(fdf_file["Bureau of Meteorology station number"].unique()[0])
col_name

'ROCKHAMPTON_MaxT_39083'

<font color = 'purple'><font size = 4>
    udf = "df for use", just the temperature vs date dataset with the column heading identifying the station name, min or max temp and BoM station ID

In [15]:
udf_use = fdf_file[['Maximum temperature (Degree C)']].copy()
udf_use.columns = [col_name]
udf_use.head()

,ROCKHAMPTON_MaxT_39083
Date,
1939-01-01,NaN
1939-01-02,NaN
1939-01-03,NaN
1939-01-04,NaN
1939-01-05,NaN


In [40]:
tempdf = udf_use.copy()

<font color = 'purple'><font size = 4>
    using the udf for one city/temperature as the starting point, the below code loops through the cleaning/processing steps above and merges into a single df

In [42]:
%%time
tempdf = pd.DataFrame(index=pd.date_range(start='2008-01-01', end='2008-12-31'))
for file in tcsvs:

    name = file.split('/')[-1].split(".")[0]

    fdf_file = pd.read_csv(file)

    fdf_file['Date'] = fdf_file.apply(func=(lambda row: pd.to_datetime(f"{row['Year']}-{row['Month']}-{row['Day']}")     ) ,axis=1)

    fdf_file.set_index('Date', drop=True, inplace=True)
    
    if 'Maximum temperature (Degree C)' in fdf_file.columns:
        udf2_use = fdf_file[['Maximum temperature (Degree C)']].copy()
    elif 'Minimum temperature (Degree C)' in fdf_file.columns:
        udf2_use = fdf_file[['Minimum temperature (Degree C)']].copy()
    udf2_use.columns = [name]# + "_" + str(fdf_file["Bureau of Meteorology station number"].unique()[0])]
    
    tempdf = tempdf.merge(udf2_use, how='outer', left_index=True, right_index=True)
                                                                       
tempdf

CPU times: user 33.3 s, sys: 296 ms, total: 33.6 s
Wall time: 34 s


In [43]:
tempdf.head()

,ARMIDALE_MinT_56037,CAPE-NELSON_MaxT_90184,TOWNSVILLE_MinT_32040,MORWELL_MaxT_85280,BRISBANE_MinT_40913,MELBOURNE_MinT_86338,SYDNEY_MaxT_66212,ADELAIDE_MaxT_23000,CANBERRA_MaxT_70351,HOBART_MinT_94029,...,MELBOURNE_MaxT_86338,BRISBANE_MaxT_40913,MORWELL_MinT_85280,SYDNEY_MinT_66212,ADELAIDE_MinT_23000,CANBERRA_MinT_70351,HOBART_MaxT_94029,LAUNCESTON_MinT_91237,PORT-AUGUSTA_MinT_18201,DUBBO_MinT_65070
1882-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1882-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1882-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1882-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1882-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
tempdf.shape

(50076, 27)

In [30]:
tempdf = tempdf[tempdf.index >= pd.to_datetime('2008-01-01')]

In [31]:
tempdf.shape

(4056, 56)

In [32]:
tempdf.index.day

Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10,
            ...
            29, 30, 31,  1,  2,  3,  4,  5,  6,  7],
           dtype='int64', name='Date', length=4056)

In [35]:
try:
    tempdf.insert(0, 'DayofMonth', tempdf.index.day)
except:
    pass
tempdf.head()

,DayofMonth,ROCKHAMPTON_MaxT_39083,ARMIDALE_MinT_56037_x,CAPE-NELSON_MaxT_90184_x,TOWNSVILLE_MinT_32040_x,MORWELL_MaxT_85280_x,BRISBANE_MinT_40913_x,MELBOURNE_MinT_86338_x,SYDNEY_MaxT_66212_x,ADELAIDE_MaxT_23000_x,...,MELBOURNE_MaxT_86338_y,BRISBANE_MaxT_40913_y,MORWELL_MinT_85280_y,SYDNEY_MinT_66212_y,ADELAIDE_MinT_23000_y,CANBERRA_MinT_70351_y,HOBART_MaxT_94029_y,LAUNCESTON_MinT_91237_y,PORT-AUGUSTA_MinT_18201_y,DUBBO_MinT_65070_y
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-01,1,32.3,13.1,19.1,23.0,39.1,21.1,NaN,NaN,NaN,...,NaN,26.8,17.2,NaN,NaN,NaN,18.0,16.0,24.7,15.5
2008-01-02,2,30.9,12.1,18.7,24.2,26.8,21.3,NaN,NaN,NaN,...,NaN,27.4,14.7,NaN,NaN,NaN,20.9,14.1,21.0,17.3
2008-01-03,3,30.7,12.2,20.0,26.0,23.1,20.8,NaN,NaN,NaN,...,NaN,25.7,16.8,NaN,NaN,NaN,21.2,16.4,20.8,16.8
2008-01-04,4,31.1,12.7,38.0,25.8,29.5,17.6,NaN,NaN,NaN,...,NaN,25.9,15.3,NaN,NaN,NaN,31.0,12.9,20.9,15.7
2008-01-05,5,33.9,14.9,27.6,22.1,35.8,20.6,NaN,NaN,NaN,...,NaN,31.0,13.6,NaN,NaN,NaN,35.1,15.3,21.1,15.3


In [36]:
tempdf.insert(1, 'isWeekend', (tempdf.index.weekday >= 5).astype(int))
tempdf.head(2)

,DayofMonth,isWeekend,ROCKHAMPTON_MaxT_39083,ARMIDALE_MinT_56037_x,CAPE-NELSON_MaxT_90184_x,TOWNSVILLE_MinT_32040_x,MORWELL_MaxT_85280_x,BRISBANE_MinT_40913_x,MELBOURNE_MinT_86338_x,SYDNEY_MaxT_66212_x,...,MELBOURNE_MaxT_86338_y,BRISBANE_MaxT_40913_y,MORWELL_MinT_85280_y,SYDNEY_MinT_66212_y,ADELAIDE_MinT_23000_y,CANBERRA_MinT_70351_y,HOBART_MaxT_94029_y,LAUNCESTON_MinT_91237_y,PORT-AUGUSTA_MinT_18201_y,DUBBO_MinT_65070_y
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-01,1,0,32.3,13.1,19.1,23.0,39.1,21.1,NaN,NaN,...,NaN,26.8,17.2,NaN,NaN,NaN,18.0,16.0,24.7,15.5
2008-01-02,2,0,30.9,12.1,18.7,24.2,26.8,21.3,NaN,NaN,...,NaN,27.4,14.7,NaN,NaN,NaN,20.9,14.1,21.0,17.3


In [37]:
tempdf.insert(0, 'Month', tempdf.index.month)
tempdf.head(2)

,Month,DayofMonth,isWeekend,ROCKHAMPTON_MaxT_39083,ARMIDALE_MinT_56037_x,CAPE-NELSON_MaxT_90184_x,TOWNSVILLE_MinT_32040_x,MORWELL_MaxT_85280_x,BRISBANE_MinT_40913_x,MELBOURNE_MinT_86338_x,...,MELBOURNE_MaxT_86338_y,BRISBANE_MaxT_40913_y,MORWELL_MinT_85280_y,SYDNEY_MinT_66212_y,ADELAIDE_MinT_23000_y,CANBERRA_MinT_70351_y,HOBART_MaxT_94029_y,LAUNCESTON_MinT_91237_y,PORT-AUGUSTA_MinT_18201_y,DUBBO_MinT_65070_y
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-01,1,1,0,32.3,13.1,19.1,23.0,39.1,21.1,NaN,...,NaN,26.8,17.2,NaN,NaN,NaN,18.0,16.0,24.7,15.5
2008-01-02,1,2,0,30.9,12.1,18.7,24.2,26.8,21.3,NaN,...,NaN,27.4,14.7,NaN,NaN,NaN,20.9,14.1,21.0,17.3


In [38]:
tempdf.insert(0,'Year',tempdf.index.year)
tempdf.head(2)

,Year,Month,DayofMonth,isWeekend,ROCKHAMPTON_MaxT_39083,ARMIDALE_MinT_56037_x,CAPE-NELSON_MaxT_90184_x,TOWNSVILLE_MinT_32040_x,MORWELL_MaxT_85280_x,BRISBANE_MinT_40913_x,...,MELBOURNE_MaxT_86338_y,BRISBANE_MaxT_40913_y,MORWELL_MinT_85280_y,SYDNEY_MinT_66212_y,ADELAIDE_MinT_23000_y,CANBERRA_MinT_70351_y,HOBART_MaxT_94029_y,LAUNCESTON_MinT_91237_y,PORT-AUGUSTA_MinT_18201_y,DUBBO_MinT_65070_y
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-01,2008,1,1,0,32.3,13.1,19.1,23.0,39.1,21.1,...,NaN,26.8,17.2,NaN,NaN,NaN,18.0,16.0,24.7,15.5
2008-01-02,2008,1,2,0,30.9,12.1,18.7,24.2,26.8,21.3,...,NaN,27.4,14.7,NaN,NaN,NaN,20.9,14.1,21.0,17.3


<font color = 'purple'><font size = 4>
    then dump output into a csv file for later use

In [39]:
dttoday = pd.to_datetime('today').strftime("%Y%m%d")
tempdf.to_csv(f"{dttoday}_28_temps_processed.csv")